In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.io
# import wfdb
import wfdb.io
from matplotlib import pyplot as plt
from tqdm import tqdm

----

In [2]:
crsidlab_root = Path('/home/thulio/projects/masters-research/CRSIDLab_Setembro_2024')

In [3]:
dataset_root = Path('/home/thulio/projects/masters-research/data')

In [4]:
relevant_signals = ['II', 'PLETH', 'ABP']

In [5]:
df_subjects = pd.read_csv('csvs/subjects.csv', index_col=0)
df_subjects

,subject_id,record_start,segment_id,master_id,class
838,92381,2150-11-15 22:32:00,3213238_0070,p092381-2150-11-15-22-32,control
396,56038,2125-09-07 22:45:00,3688191_0033,p056038-2125-09-07-22-45,control
740,83263,2145-12-08 23:07:00,3461856_0013,p083263-2145-12-08-23-07,control
78,57752,2109-06-02 21:27:00,3269067_0003,p057752-2109-06-02-21-27,control
386,73471,2125-07-11 14:44:00,3570060_0007,p073471-2125-07-11-14-44,control
802,55044,2150-09-09 16:26:00,3487905_0003,p055044-2150-09-09-16-26,control
204,31574,2114-12-28 09:31:00,3728524_0007,p031574-2114-12-28-09-31,control
208,28775,2116-03-04 18:52:00,3872045_0004,p028775-2116-03-04-18-52,control
1203,67659,2179-07-06 01:15:00,3469659_0027,p067659-2179-07-06-01-15,control
640,85980,2140-09-16 14:33:00,3455932_0013,p085980-2140-09-16-14-33,control


In [6]:
positions = [
    77695, # 1
    0, # 2 X
    642831, # 3
    852839, # 4
    174292, # 5 ?
    265749, # 6 ?
    112562, # 7
    477844, # 8
    9979, # 9
    173314, # 10
    83047, # 11
    3661915, # 12
    257967, # 13
    498066, # 14
    0, # 15 X
    149224, # 16
    203196, # 17
    3814737, # 18
    127229, # 19
    2434134 # 20
]

In [7]:
window = 5 * 60

In [ ]:
subjects: list[tuple[int, str, str, str, str]] = df_subjects.itertuples(index=False)
for (subject_id, record_start, segment_id, master_id, label), pos in tqdm(zip(subjects, positions)):
    p_folder = dataset_root / label / f'p{subject_id:06d}'
    # master_header = wfdb.io.rdheader((p_folder / master_id).as_posix().replace('.hea', ''), rd_segments=True)
    # segment: wfdb.io.Record = master_header.segments[master_header.seg_name.index(segment_id)]
    w_end = pos + fields['fs'] * window
    w_start = pos
    signals, fields = wfdb.io.rdsamp(p_folder / segment_id, sampfrom=w_start, sampto=w_end, channel_names=relevant_signals)
    patient_file = crsidlab_root / 'subjects' / f'{p_folder.name}_{segment_id}.mat'
    mat_data = {
        'fs': fields['fs'],
        'timeVec': np.arange(fields['sig_len']) / fields['fs'],
        'ecg': signals[:, 0],
        'pleth': signals[:, 1],
        'abp': signals[:, 2],
        # 'resp': signals[:, 3],
    }
    scipy.io.savemat(patient_file, mat_data)

0it [00:00, ?it/s]


NameError: name 'fields' is not defined